In [9]:
import pysam as ps
import numpy as np
import operator
import random
import timeit

In [10]:
tbx_glf = ps.TabixFile("loglikes.glf.gz")
tbx_mafs = ps.TabixFile("loglikes.mafs.gz")

In [20]:
start_time = timeit.default_timer()

region = 4e+6
# Chromesome list excluding x and y chromesomes
chr_ = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]

# Array of number of bp in each crhomesome, from 1 - 22, minus region of choice
chr_bp = np.array((248956422, 242193529, 198295559, 190214555, 181538259,
         170805979, 159345973, 145138636, 138394717, 133797422,
         135086622, 133275309, 114364328, 107043718, 101991189,
         90338345, 83257441, 80373285, 58617616, 64444167,
         46709983, 50818468)) - region

# Chooses random chromesome and random starting position for the region
chr_nr = random.randint(0, len(chr_)-1)
start_pos = random.randint(0, chr_bp[chr_nr])

# Indexing in the gz files via tabix through pysam
glf = []
mafs = []

for i in tbx_glf.fetch(chr_nr+1, start_pos, start_pos+region):
     glf.append(i.split('\t'))

for i in tbx_mafs.fetch(chr_nr+1, start_pos, start_pos+region):
     mafs.append(i.split('\t'))

elapsed = timeit.default_timer() - start_time
elapsed/60

0.22321010478772224

In [21]:
start_time = timeit.default_timer()

glf = np.float_(glf)

elapsed = timeit.default_timer() - start_time
elapsed/60

0.37521033418985705

In [22]:
start_time = timeit.default_timer()

N = []
for i in glf:
    N.append(np.array_split(i[2:], (len(i)-2)/10))

elapsed = timeit.default_timer() - start_time
elapsed/60

0.9659607757503788

In [23]:
start_time = timeit.default_timer()

gl_mat = []
genomic_pos = []
rand_N = random.randint(0,len(N[0])-1)
for i, val in enumerate(N):
    if mafs[i][2] == 'A' and mafs[i][3] == 'C' or mafs[i][2] == 'C' and mafs[i][3] == 'A':
        gl_mat.append(np.array(operator.itemgetter(0,1,4)(val[rand_N])))
        genomic_pos.append(glf[i][1])
    elif mafs[i][2] == 'A' and mafs[i][3] == 'G' or mafs[i][2] == 'G' and mafs[i][3] == 'A':
        gl_mat.append(np.array(operator.itemgetter(0,2,7)(val[rand_N])))
        genomic_pos.append(glf[i][1])
    elif mafs[i][2] == 'A' and mafs[i][3] == 'T' or mafs[i][2] == 'T' and mafs[i][3] == 'A':
        gl_mat.append(np.array(operator.itemgetter(0,3,9)(val[rand_N])))
        genomic_pos.append(glf[i][1])
    elif mafs[i][2] == 'C' and mafs[i][3] == 'G' or mafs[i][2] == 'G' and mafs[i][3] == 'C':
        gl_mat.append(np.array(operator.itemgetter(4,5,7)(val[rand_N])))
        genomic_pos.append(glf[i][1])
    elif mafs[i][2] == 'C' and mafs[i][3] == 'T' or mafs[i][2] == 'T' and mafs[i][3] == 'C':
        gl_mat.append(np.array(operator.itemgetter(4,6,9)(val[rand_N])))
        genomic_pos.append(glf[i][1])
    elif mafs[i][2] == 'G' and mafs[i][3] == 'T' or mafs[i][2] == 'T' and mafs[i][3] == 'G':
        gl_mat.append(np.array(operator.itemgetter(7,8,9)(val[rand_N])))
        genomic_pos.append(glf[i][1])
    else:
        print('ERROR')

elapsed = timeit.default_timer() - start_time
elapsed/60

0.08047320110102495